In [54]:
import cv2
import numpy as np


cap = cv2.VideoCapture(0)

_,face_frame = cap.read()
  
face_cascade = cv2.CascadeClassifier('Images/haarcascades/haarcascade_frontalface_alt2.xml')

face_rets = face_cascade.detectMultiScale(face_frame)
(x,y,w,h) = tuple(face_rets[0])
track_window = (x,y,w,h)
roi = face_frame[y:y+h,x:x+w]
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])
roi_hist = cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
term_critera = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)            
                
while True:
    _,frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    mask = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
    
    
    rets, track_window = cv2.CamShift(mask, track_window, term_criteria)
    pts = cv2.boxPoints(rets)
    pts = np.int0(pts)
    img2 = cv2.polylines(frame,[pts],True,(0,255,0),3)
    
    cv2.imshow('Frame',img2)
    cv2.imshow('Mask',mask)
    if cv2.waitKey(1)== 27:
        break
        
cap.release()
cv2.destroyAllWindows()          